In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
from fastai import *
from fastai.text import *
import torch
import torch.nn as nn
from nlputils import split_wiki,get_wiki
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = Config.data_path()
lang = 'vi'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
bs=64

In [3]:
train_df = pd.read_csv('data/train.csv', sep='\t')
train_df.loc[pd.isna(train_df.text),'text']='NA'
train_df

,id,text,label
0,train_000000,Dung dc sp tot cam on shop Đóng gói sản phẩm r...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng ...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp k...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1
...,...,...,...
16082,train_016082,Chẳng biết là Shop có biết đọc hay không mua ố...,1
16083,train_016083,Cuốn này mỏng. Đọc một buổi sáng là hết. Thú t...,1
16084,train_016084,Mang êm chân. Đẹp,0
16085,train_016085,Tôi đã nhận đc hàng.Sau đây là vài lời muốn nó...,1


In [4]:
test_df = pd.read_csv('data/test.csv', sep='\t')
test_df.loc[pd.isna(test_df.text),'text']='NA'
test_df

,id,text
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua nh...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp
...,...,...
10976,test_010976,Thời gian giao hàng rất nhanh.ngon.mà cay quá....
10977,test_010977,Sản phẩm hơi cũ
10978,test_010978,Sản phẩm chắc chắn nhưng k bóng bằng trong hình
10979,test_010979,Chất lượng sản phẩm tuyệt vời có mùi thơm rất ...


In [5]:
df = pd.concat([train_df,test_df], sort=False)
df

,id,text,label
0,train_000000,Dung dc sp tot cam on shop Đóng gói sản phẩm r...,0.0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng ...,0.0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp k...,0.0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1.0
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1.0
...,...,...,...
10976,test_010976,Thời gian giao hàng rất nhanh.ngon.mà cay quá....,NaN
10977,test_010977,Sản phẩm hơi cũ,NaN
10978,test_010978,Sản phẩm chắc chắn nhưng k bóng bằng trong hình,NaN
10979,test_010979,Chất lượng sản phẩm tuyệt vời có mùi thơm rất ...,NaN


In [6]:
data_lm = (TextList.from_df(df, path, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=4))

In [7]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [8]:
lr = 1e-3
lr *= bs/48

In [9]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.766068,4.148816,0.319852,00:09
1,4.282370,4.023852,0.329129,00:09


In [10]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.143759,3.940941,0.335658,00:12
1,4.006675,3.815681,0.347879,00:13
2,3.847452,3.710678,0.356306,00:12
3,3.753218,3.644303,0.363239,00:12
4,3.678279,3.599177,0.368290,00:12
5,3.620201,3.570790,0.371080,00:13
6,3.558057,3.550967,0.372070,00:12
7,3.526611,3.539626,0.373256,00:13
8,3.487714,3.536422,0.373549,00:13
9,3.501466,3.536272,0.373758,00:13


In [11]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

In [12]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))
data_clas.save(f'{lang}_textlist_class')

In [13]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
data_clas

TextClasDataBunch;

Train: LabelList (14479 items)
x: TextList
xxbos xxmaj dung dc sp tot cam on shop xxmaj đóng gói sản phẩm rất đẹp và chắc chắn xxmaj chất lượng sản phẩm tuyệt vời,xxbos xxmaj chất lượng sản phẩm tuyệt vời . xxmaj son mịn nhưng khi đánh lên không như màu trên ảnh,xxbos xxmaj chất lượng sản phẩm tuyệt vời nhưng k có hộp k có dây giày đen k có tất,xxbos xxmaj lần trước mình mua áo gió màu hồng rất ok mà đợt này lại giao 2 cái áo gió chất khác như vải mưa ý :((,xxbos xxmaj chất lượng sản phẩm tuyệt vời có điều không cứng cáp với không cố định dáng nói chung đẹp hợp túi tiền
y: CategoryList
0,0,0,1,0
Path: /home/tuna/.fastai/data/viwiki;

Valid: LabelList (1608 items)
x: TextList
xxbos xxmaj uống xong 1 tuần thấy ng nhẹ hẳn . xxmaj phải xuống 3 kg . m đã dùng xxunk nhưng giảm rất ít nên chuyển qua xxunk thấy ok .,xxbos xxmaj đóng gói sản phẩm rất kém - chảy ướt cả hộp giấy gói xxmaj rất không đáng tiền,xxbos xxmaj fb cho shop nè 😂 😂 😂,xxbos xxmaj giao hàng nhanh theo đún

In [14]:
from sklearn.metrics import f1_score
@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [15]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [16]:
lr=2e-2
lr *= bs/48

In [17]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.352346,0.286327,0.875622,0.854120,00:03
1,0.315573,0.255784,0.898010,0.873664,00:03


In [18]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.334256,0.287082,0.885572,0.844787,00:03
1,0.304901,0.260951,0.889925,0.865925,00:03


In [19]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.303741,0.257199,0.899254,0.869795,00:04
1,0.256774,0.223866,0.904229,0.878354,00:04


In [20]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.241180,0.247208,0.901741,0.884843,00:06
1,0.192413,0.224999,0.908582,0.884098,00:06


In [21]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.156582,0.241357,0.903607,0.878886,00:08


In [22]:
learn_c.save(f'{lang}clas')

In [23]:
preds,targs = learn_c.get_preds(ordered=True)
print(preds)
print(targs)

tensor([[8.7016e-01, 1.2984e-01],
        [2.3597e-04, 9.9976e-01],
        [9.9385e-01, 6.1451e-03],
        ...,
        [9.9774e-01, 2.2562e-03],
        [9.8583e-01, 1.4172e-02],
        [2.5810e-02, 9.7419e-01]])
tensor([0, 1, 0,  ..., 0, 0, 1])


In [24]:
targs.shape

torch.Size([1608])

In [25]:
preds.shape

torch.Size([1608, 2])

In [40]:
a = learn_c.predict(':(( Mình hơi thất vọng 1 chút')
a

(Category tensor(1), tensor(1), tensor([0.2703, 0.7297]))

In [53]:
a[2][0].item()

0.27028757333755493